# FEM Solution of 1-D transient heat transfer in a rod with composite material and internal heat generation


![](Figure.jpg)


- **Ambient temperature** (Constant, Constant rate, One hold temperature cycle)
        
- **Boundary Condition** (Convection heat transfer)
        
- **Solutiion method** (Explicit Euler, Implicit Euler w/ Newton Raphson)
            
- **Material** (AS4/3501-6 Composite, Invar Tool)
            
- **Mesh** (Uniform)
            
- **Element** (2-Node Linear, 3-Node Nonlinear)

In [1]:
# import 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patches as patches
import scipy as sp
from scipy import linalg
from ipywidgets import interactive
from IPython.display import clear_output
#%matplotlib inline

In [2]:
#set figure defaults for IPython notebook (must be in separate cell from above)
#matplotlib.rcParams.update({'font.size': 18, 'lines.linewidth':4})

## Main functions

In [3]:
# Functions 
# Air temperature function, Constant, Constant Rate, OneHold
def T_air(t,typ,T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma):
    # air_temp_type, 'Constant', 'Constant_rate', 'OneHold'
    # T_start, start air temperature (K)
    # T_const, air consat temperate (for 'Constat' type) (K)
    # T_rate, air temperature increase rate (for 'Constant_rate' type)
    # T_hold, air hold temperate (for 'OneHold' type) (K)
    # th1, time for start of hold (for 'OneHold' type) (seconds)
    # th2, time for end of hold (for 'OneHold' type) (seconds)
    if typ == 'ConstantRate':
        T_a = np.random.normal(t*T_rate + T_start,T_air_sigma) # t*T_rate + T_start + np.random.normal(0,T_air_sigma)
    elif typ == 'OneHold':
        T_rate = (T_hold-T_start) / th1
        if t < th1:
            T_a =  np.random.normal(t*T_rate + T_start,T_air_sigma) # t*T_rate + T_start + np.random.normal(0,T_air_sigma)
        elif t > th2:
            T_a = -(t-th2)*T_rate + T_hold + np.random.normal(0,T_air_sigma)
        else:
            T_a = np.random.normal(T_hold,T_air_sigma) # T_hold + np.random.normal(0,T_air_sigma)
    elif typ == 'Constant':
        T_a = np.random.normal(T_const,T_air_sigma) # T_const + np.random.normal(0,T_air_sigma)
    return T_a


# Material model
# C Coefficients
def C(Temp,A1,A2,A3,dE1,dE2,dE3):
    A = [A1, A2, A3]
    dE = [dE1, dE2, dE3]
    R = 8.314
    C = list(range(3))
    for i in range(3):
        C[i]= A[i] * np.exp(-dE[i]/(R*Temp))
    return C



# Rate of degree of cure function 
def alpha_dot_func(alpha,T,BB,A1,A2,A3,dE1,dE2,dE3):  
    if alpha <= 0.3:
        alpha_dot = (C(T,A1,A2,A3,dE1,dE2,dE3)[0]+(C(T,A1,A2,A3,dE1,dE2,dE3)[1]*alpha)) * (1-alpha) * (BB-alpha)
    else:
        alpha_dot = C(T,A1,A2,A3,dE1,dE2,dE3)[2]*(1-alpha) 
    return alpha_dot

# Degree of cure
def alpha_func(el,t_start,t_end,delt,T_sol,Element,BB,A1,A2,A3,dE1,dE2,dE3):
    alpha_arr = np.array([0])
    alpha_dot_arr = np.array([0])
    for t in np.arange(t_start+delt,t_end,delt):
        if Element == 'Linear':
            T_el = 0.5*(T_sol[el-1,int((t-t_start)/delt)]+T_sol[el,int((t-t_start)/delt)])
        elif Element == 'Nonlinear':
            T_el = (T_sol[2*el-1,int((t-t_start)/delt)]) # center node
            
        alpha_dot = alpha_dot_func(alpha_arr[-1],T_el,BB,A1,A2,A3,dE1,dE2,dE3)
        alpha_arr = np.append(alpha_arr, alpha_arr[-1] + delt * alpha_dot)
        alpha_dot_arr = np.append(alpha_dot_arr, alpha_dot)
    return alpha_arr, alpha_dot_arr


# Arrays carrying information about the mesh, 2 node elements
def Mesh(Coords_start,Length_c,num_el_c,Length_t,num_el_t):    
    num_el = num_el_c + num_el_t
    Length = Length_c + Length_t
    Nodes = np.linspace(1,num_el+1,num_el+1)
    Elements= np.zeros((num_el,2))
    Elements[:,0] = np.linspace(1,num_el,num_el)
    Elements[:,1] = np.linspace(2,num_el+1,num_el)
    #Coords = np.linspace(Coords_start,Coords_start+Length,num_el+1)
    Coords_t = np.linspace(Coords_start,Coords_start+Length_t,num_el_t+1)
    Coords_c = np.linspace(Coords_start+Length_t,Coords_start+Length_t+Length_c,num_el_c+1)
    #Coords = Coords_t + Coords_c[1:]
    Coords = np.concatenate((Coords_t, Coords_c[1:]))
    return Nodes, Elements, Coords

# Arrays carrying information about the mesh, 3 node elements
def Mesh3(Coords_start,Length_c,num_el_c,Length_t,num_el_t):    
    num_el = num_el_c + num_el_t
    Length = Length_c + Length_t    
    Nodes = np.linspace(1,2*num_el+1,2*num_el+1)
    Elements= np.zeros((num_el,3))
    Elements[:,0] = np.linspace(1,2*num_el-1,num_el)
    Elements[:,1] = np.linspace(2,2*num_el,num_el)
    Elements[:,2] = np.linspace(3,2*num_el+1,num_el)
    Coords = np.linspace(Coords_start,Coords_start+Length,2*num_el+1)
    return Nodes, Elements, Coords

# Element K, C, C_lumped, and F matrixes, 2 node element
def KCF(el,a_c,b_c,a_t,b_t,alpha_dot_el,Coords):    
    if el > num_el_t:
        a = a_c
        b = b_c
    else:
        a = a_t
        b = b_t
        
    L = Coords[el] - Coords[el-1] # element length    
    K_el = np.zeros((2,2))
    C_el = np.zeros((2,2))
    F_el = np.zeros(2)
    for kesi in [-1/(3**0.5), 1/(3**0.5)]:  # 2 gauss points      
        # shpae functions
        N = np.array([(1-kesi)/2, (1+kesi)/2])
        B = 0.5 * np.array([-1, 1])        
        K_el += (2/L) * a * np.outer(B,B)
        C_el += (L/2) * np.outer(N,N)
        F_el = (L/2) * alpha_dot_el * b * N 
    C_lumped_el = L/2 * np.array([[1, 0],[0, 1]]) 
    return K_el, C_el, C_lumped_el, F_el

# Element K, C, C_lumped, and F matrixes, 3 node element
def KCF3(el,a_c,b_c,a_t,b_t,alpha_dot_el,Coords):    
    if el > num_el_t:
        a = a_c
        b = b_c
    else:
        a = a_t
        b = b_t
    
    L = Coords[2*el] - Coords[2*el-2] # element length    
    K_el = np.zeros((3,3))
    C_el = np.zeros((3,3))
    F_el = np.zeros(3)
    i = 0
    weight = [5/9, 8/9, 5/9]
    for kesi in [-((3/5)**0.5), 0, ((3/5)**0.5)]:  # 3 gauss points      
        # shpae functions
        N = np.array([kesi*(kesi-1)/2, 1-kesi**2, kesi*(kesi+1)/2]) * weight[i]
        B = np.array([kesi-0.5, -2*kesi, kesi+0.5])* weight[i]        
        K_el += (2/L) * a * np.outer(B,B)
        C_el += (L/2) * np.outer(N,N)
        F_el = (L/2) * alpha_dot_el * b * N    
        i += 1
    C_lumped_el = L/2 * np.array([[1, 0, 0],[0, 1, 0],[0, 0, 1]]) 
    return K_el, C_el, C_lumped_el, F_el

# Assembly of element array into global arrays, 2 node elements
def Assemble(el,K_el,C_el,C_lumped_el,F_el,K_sys,C_sys,C_lumped_sys,F_sys):    
    ii = el-1
    for i in range(2):
        F_sys[ii] += F_el[i]
        jj = el-1
        for j in range(2):
            K_sys[ii][jj] += K_el[i][j]
            C_sys[ii][jj] += C_el[i][j]
            C_lumped_sys[ii][jj] += C_lumped_el[i][j]
            jj += 1
        ii += 1 
    return K_sys, C_sys, C_lumped_sys, F_sys

# Assembly of element array into global arrays, 3 node elements
def Assemble3(el,K_el,C_el,C_lumped_el,F_el,K_sys,C_sys,C_lumped_sys,F_sys):    
    ii = 2*el-2
    for i in range(3):
        F_sys[ii] += F_el[i]
        jj = 2*el-2
        for j in range(3):
            K_sys[ii][jj] += K_el[i][j]
            C_sys[ii][jj] += C_el[i][j]
            C_lumped_sys[ii][jj] += C_lumped_el[i][j]
            jj += 1
        ii += 1 
    return K_sys, C_sys, C_lumped_sys, F_sys


## Visualization functions

In [4]:
# Temperature plots
def plot_Temp(t,t_start,delt,T_ini,Analysis,Coords,T_sol,air_temp_type,num_el_c,num_el_t, a_c, b_c,Ch_c):
    # Temperature distribution along the rod at specific instance of t
    plt.figure(figsize=(10,10))   
    plt.subplot(211)
    plt.scatter(Coords,T_sol[:,int((t-t_start)/delt)])
    plt.plot(Coords[0:num_el_t+1],T_sol[0:num_el_t+1,int((t-t_start)/delt)], color='green')
    plt.plot(Coords[num_el_t:],T_sol[num_el_t:,int((t-t_start)/delt)])
    plt.xlabel('x location (m)')
    plt.ylabel('Temperature (K)')
    plt.ylim(round(T_ini[0,0])-1,round(np.amax(T_sol))*1.1)
    plt.title('delt = {} s,  Analysis : {}, Element: {}, Heat generation: {}\n a_c = {}, b_c ={}, Ch_c ={}'.
              format(delt,Analysis,Element_type,heat_gen,a_c,b_c,Ch_c, fontsize=14, loc="left"))
    plt.show()
    
    # Air temperature evolution
    plt.figure(figsize=(10,10))
    plt.subplot(212)
    T_air_arr = np.array([T_air(0,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma)])
    for time in np.arange(t_start+delt,t_end,delt):
        T_air_arr = np.append(T_air_arr, T_air(time,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma))         
    plt.plot(np.arange(t_start,t_end,delt),T_air_arr)
    plt.plot([t,t],[round(np.amax(T_air_arr)),round(np.amax(T_air_arr))*1.1], color = 'red')
    plt.xlabel('Time (s)')
    plt.ylabel('Air temperature (K)')
    plt.ylim(round(T_ini[0,0])-1,round(np.amax(T_air_arr))*1.1)
    plt.show()


In [5]:
# Temperature plots
def plot_alpha(t,t_start,delt,alpha_ini,Analysis,Coords,alpha_sol,air_temp_type):
    # Temperature distribution along the rod at specific instance of t
    Coords_el = Coords[:-1] + Length/num_el
    plt.figure(figsize=(10,10))   
    plt.subplot(211)
    plt.scatter(Temperature,alpha_sol[:,int((t-t_start)/delt)])
    plt.plot(Temperature,alpha_sol[:,int((t-t_start)/delt)])
    plt.xlabel('x location (m)')
    plt.ylabel('Degree of cure')
    plt.ylim(round(alpha_ini[0,0])-1,round(np.amax(alpha_sol))*1.1)
    plt.title('delt = {} s,  Analysis : {}, Element: {}, Heat generation: {}\n a = {}, b={}, Ch ={}'.
              format(delt,Analysis,Element_type,heat_gen,a,b,Ch), fontsize=14, loc="left")
    plt.show()
    
    # Air temperature evolution
    plt.figure(figsize=(10,10))
    plt.subplot(212)
    T_air_arr = np.array([T_air(0,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma)])
    for time in np.arange(t_start+delt,t_end,delt):
        T_air_arr = np.append(T_air_arr, T_air(time,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma))         
    plt.plot(np.arange(t_start,t_end,delt),T_air_arr)
    plt.plot([t,t],[round(np.amax(T_air_arr)),round(np.amax(T_air_arr))*1.1], color = 'red')
    plt.xlabel('Time (s)')
    plt.ylabel('Air temperature (K)')
    plt.ylim(round(T_ini[0,0])-1,round(np.amax(T_air_arr))*1.1)
    plt.show()

In [6]:
# Elements temperature and degree of cure plots
def plot_T_alpha_el(el,t_start,t_end,delt,T_sol,alpha_sol,alpha_dot_sol,air_temp_type):    
    # Degree of cure for element el
    plt.figure(figsize=(10,15))  
    plt.subplot(311)
    plt.plot(np.arange(t_start,t_end,delt),alpha_sol[el-1,:])
    plt.xlabel('Time (s)')
    plt.ylabel('Element degree of cure')
    plt.ylim(0,np.amax(alpha_sol[el-1,:])*1.1)
    plt.show()     
    plt.figure(figsize=(10,15))  
    plt.subplot(312)
    plt.plot(np.arange(t_start,t_end,delt),alpha_dot_sol[el-1,:])
    plt.xlabel('Time (s)')
    plt.ylabel('Element rate of degree of cure')
    plt.ylim(0,np.amax(alpha_dot_sol[el-1,:])*1.1)
    plt.show()
    
    # Element el temperature evolution
    plt.figure(figsize=(10,15))
    plt.subplot(313)
    
    if Element_type == 'Linear':
        T_el_arr = 0.5*(T_sol[el-1,:]+T_sol[el,:])
    elif Element_type == "Nonlinear":
        T_el_arr = (T_sol[2*el-1,:]) # Center node
        
    plt.plot(np.arange(t_start,t_end,delt),T_el_arr, label="element")
    T_air_arr = np.array([T_air(0,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma)])
    for time in np.arange(t_start+delt,t_end,delt):
        T_air_arr = np.append(T_air_arr, T_air(time,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma))         
    plt.plot(np.arange(t_start,t_end,delt),T_air_arr, label="air")
    plt.legend(loc='best')
    plt.xlabel('Time(s)')
    plt.ylabel('Temperature (K)')
    plt.show()
    


In [7]:
def plot_temp_error_shade(t,t_start,t_end,delt,Coords,T_mean,T_var,air_temp_type,
                          T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma,T_ini,
                          sensor_loc_typ,sensor_loc_list,Length_c,Length_t,num_el_c,num_el_t):
    Length = Length_c + Length_t
    num_el = num_el_c + num_el_t
    # vizualizing mean and variacen of temperature for the rod    
    if sensor_loc_typ == "node":
        sensor_loc_n = sensor_loc_list # a list, node numbers
        sensor_loc = [min((i-1),num_el_t) * (Length_t/num_el_t) +
                      max(0, (i-1-num_el_t)) * (Length_c/num_el_c) for i in sensor_loc_n] 
    elif sensor_loc_typ == "loc":
        sensor_loc = sensor_loc_list # a list, location of sensor (m)
        #sensor_loc_n = [int(round(x /  (Length/num_el))) + 1 for x in sensor_loc] # sensor location node number
        sensor_loc_n = [int(min(num_el_t, round( x / (Length_t / num_el_t)) ) +
                            max(0, round( (x-Length_t) /  (Length_c/num_el_c))) ) + 1 for x in sensor_loc] # sensor location node number

    tn = int((t-t_start)/delt)
    plt.figure(figsize=(10,10))  
    plt.subplot(311)
    plt.fill_between(Coords, T_mean[:, tn]-T_var[:, tn], T_mean[:, tn]+T_var[:, tn], alpha=0.5)
    #plt.plot(Coords, T_mean[:, tn], color = 'k', linewidth=1)
    plt.plot(Coords[0:num_el_t+1], T_mean[0:num_el_t+1, tn], color = 'red', linewidth=1)
    plt.plot(Coords[num_el_t:num_el_t+num_el_c+2], T_mean[num_el_t:num_el_t+num_el_c+2, tn], color = 'k', linewidth=1)
    
    if len(sensor_loc)>0: # if there is any sensor
        plt.plot([sensor_loc,sensor_loc],[round(np.amin(T_mean)),round(np.amax(T_mean))*1.1]
                 , color = 'green', linewidth=1)
    plt.ylim([np.amin(T_mean)-np.amax(T_var),np.amax(T_mean)+np.amax(T_var)])
    if len(sensor_loc)>0: # if there is any sensor
        plt.title('Sensor location (shown as green line)= {} m \nSensor is at node(s)  {}.'
                  .format(sensor_loc, sensor_loc_n), fontsize=14, loc="left")
    plt.xlabel('location (m)')
    plt.ylabel('Temperature (K)')
    plt.show()
    # air temperature evolution
    plt.figure(figsize=(10,10))
    plt.subplot(312)
    T_air_arr = np.array([T_air(0,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma)])
    for time in np.arange(t_start+delt,t_end,delt):
        T_air_arr = np.append(T_air_arr, T_air(time,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma))         
    plt.plot(np.arange(t_start,t_end,delt),T_air_arr)
    plt.plot([t,t],[round(np.amax(T_air_arr)),round(np.amax(T_air_arr))*1.1], color = 'red', linewidth=1)
    plt.xlabel('Time (s)')
    plt.ylabel('Air temperature (K)')
    plt.title('Current time is shown as red line', fontsize=14, loc="left")
    plt.ylim(round(T_ini[0,0])-1,round(np.amax(T_air_arr))*1.1)
    plt.show()
    # std at different nodes 
    plt.figure(figsize=(10,10)) 
    plt.subplot(313) 
    #plt.plot(Coords, T_var[:, tn], color = 'k', linewidth=1)
    plt.plot(Coords[0:num_el_t+1], T_var[0:num_el_t+1, tn], color = 'red', linewidth=1)
    plt.plot(Coords[num_el_t:num_el_t+num_el_c+2], T_var[num_el_t:num_el_t+num_el_c+2, tn], color = 'k', linewidth=1)
    
    if len(sensor_loc)>0: # if there is any sensor
        plt.plot([sensor_loc,sensor_loc],[round(np.amin(T_var)),round(np.amax(T_var))*1.1]
                 , color = 'green', linewidth=1)
    plt.xlabel('location (m)')
    plt.ylabel('Standard Deviation')
    plt.ylim([np.amin(T_var),np.amax(T_var)])
    if len(sensor_loc)>0: # if there is any sensor
        plt.title('Sensor location (shown as green line)= {} m \nSensor is at node(s)  {}.'
                  .format(sensor_loc, sensor_loc_n), fontsize=14, loc="left")
    plt.show()




In [8]:
def plot_node_temp_std(node_number,T_mean,T_var,t_start,t_end,delt,air_temp_type,
                            T_start,T_hold,T_const,T_rate,th1,th2,sensor_loc_typ,sensor_loc_list,
                            Length_c,Length_t,num_el_c,num_el_t,T_air_sigma):
    
    Length = Length_c + Length_t
    num_el = num_el_c + num_el_t
    
    if sensor_loc_typ == "node":
        sensor_loc_n = sensor_loc_list # a list, node numbers
        sensor_loc = [min((i-1),num_el_t) * (Length_t/num_el_t) +
                      max(0, (i-1-num_el_t)) * (Length_c/num_el_c) for i in sensor_loc_n]
    elif sensor_loc_typ == "loc":
        sensor_loc = sensor_loc_list # a list, location of sensor (m)
        #sensor_loc_n = [int(round(x /  (Length/num_el))) + 1 for x in sensor_loc] # sensor location node number
        sensor_loc_n = [int(min(num_el_t, round( x / (Length_t / num_el_t)) ) +
                            max(0, round( (x-Length_t) / (Length_c/num_el_c))) ) + 1 for x in sensor_loc] # sensor location node number

    plt.figure(figsize=(10,10))   
    plt.subplot(211)
    plt.plot(np.arange(t_start,t_end,delt), T_mean[node_number-1, :-1], color = 'k', linewidth=1)
    # air temperature
    T_air_arr = np.array([T_air(0,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma)])
    for time in np.arange(t_start+delt,t_end,delt):
        T_air_arr = np.append(T_air_arr, T_air(time,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma))         
    plt.plot(np.arange(t_start,t_end,delt),T_air_arr, color = 'blue', linewidth=1, alpha=0.3)
    plt.xlabel('Time (s)')
    plt.ylabel('Temperature (K)')
    plt.title('Sensor is at location= {} m \nSensor is at node(s) {}.\nBlue line is air temperature.'
              .format(sensor_loc, sensor_loc_n), fontsize=14, loc="left")
    plt.ylim([np.amin(T_mean)-np.amax(T_var),np.amax(T_mean)+np.amax(T_var)])
    plt.show()
    # std of node temperature at different time    
    plt.figure(figsize=(10,10))   
    plt.subplot(212)
    plt.plot(np.arange(t_start,t_end,delt), T_var[node_number-1, :-1], color = 'k', linewidth=1)
    plt.xlabel('Time (s)')
    plt.ylabel('Standard Deviation')
    plt.ylim([np.amin(T_var),np.amax(T_var)])
    plt.show()

In [9]:
def plot_alpha_error_shade(t,t_start,t_end,delt,Coords,alpha_mean,alpha_var,air_temp_type,
                          T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma,T_ini,
                           sensor_loc_typ,sensor_loc_list,Length_c,Length_t,num_el_c,num_el_t):

    # vizualizing mean and variacen of degree of cure for the rod 
    Length = Length_c + Length_t
    num_el = num_el_c + num_el_t    
    Coords_el = [0] * (len(Coords)-1)
    Coords_el[0:num_el_t] = Coords[0:num_el_t] + Length_t/num_el_t
    Coords_el[num_el_t:num_el_t+num_el_c] = Coords[num_el_t:num_el_t+num_el_c] + Length_c/num_el_c
    
    if sensor_loc_typ == "node":
        sensor_loc_n = sensor_loc_list # a list, node numbers
        sensor_loc = [min((i-1),num_el_t) * (Length_t/num_el_t) +
                      max(0, (i-1-num_el_t)) * (Length_c/num_el_c) for i in sensor_loc_n] 
    elif sensor_loc_typ == "loc":
        sensor_loc = sensor_loc_list # a list, location of sensor (m)
        #sensor_loc_n = [int(round(x /  (Length/num_el))) + 1 for x in sensor_loc] # sensor location node number
        sensor_loc_n = [int(min(num_el_t, round( x / (Length_t / num_el_t)) ) +
                            max(0, round( (x-Length_t) / (Length_c / num_el_c)) )) + 1 for x in sensor_loc] # sensor location node number
    
    
    tn = int((t-t_start)/delt)
    plt.figure(figsize=(10,10))  
    plt.subplot(311)
    plt.fill_between(Coords_el, alpha_mean[:, tn]-alpha_var[:, tn], alpha_mean[:, tn]+alpha_var[:, tn], alpha=0.5)
    #plt.plot(Coords_el, alpha_mean[:, tn], color = 'k', linewidth=1)
    plt.plot(Coords_el[0:num_el_t], alpha_mean[0:num_el_t, tn], color = 'red', linewidth=1)
    plt.plot(Coords_el[num_el_t:num_el_t+num_el_c+1], alpha_mean[num_el_t:num_el_t+num_el_c+1, tn], color = 'k', linewidth=1)
    
    if len(sensor_loc)>0: # if there is any sensor
        plt.plot([sensor_loc,sensor_loc],[-0.1,1.1]
                 , color = 'green', linewidth=1)
    plt.ylim([np.amin(alpha_mean)-np.amax(alpha_var),np.amax(alpha_mean)+np.amax(alpha_var)])
    if len(sensor_loc)>0: # if there is any sensor
        plt.title('Sensor location (shown as green line)= {} m \nSensor is at node(s)  {}.'
                  .format(sensor_loc, sensor_loc_n), fontsize=14, loc="left")
    plt.xlabel('location (m)')
    plt.ylabel('Degree of cure')
    plt.show()
    # air temperature evolution
    plt.figure(figsize=(10,10))
    plt.subplot(312)
    T_air_arr = np.array([T_air(0,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma)])
    for time in np.arange(t_start+delt,t_end,delt):
        T_air_arr = np.append(T_air_arr, T_air(time,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma))         
    plt.plot(np.arange(t_start,t_end,delt),T_air_arr)
    plt.plot([t,t],[round(np.amax(T_air_arr)),round(np.amax(T_air_arr))*1.1], color = 'red', linewidth=1)
    plt.xlabel('Time (s)')
    plt.ylabel('Air temperature (K)')
    plt.title('Current time is shown as red line', fontsize=14, loc="left")
    plt.ylim(round(T_ini[0,0])-1,round(np.amax(T_air_arr))*1.1)
    plt.show()
    # std at different nodes 
    plt.figure(figsize=(10,10))
    plt.subplot(313) 
    plt.plot(Coords_el[0:num_el_t], alpha_var[0:num_el_t,tn], color = 'red', linewidth=1)
    plt.plot(Coords_el[num_el_t:num_el_t+num_el_c+1], alpha_var[num_el_t:num_el_t+num_el_c+1,tn], color = 'k', linewidth=1)
    if len(sensor_loc)>0: # if there is any sensor
        plt.plot([sensor_loc,sensor_loc],[-0.5,round(np.amax(alpha_var))*1.1]
                 , color = 'green', linewidth=1)
    plt.xlabel('location (m)')
    plt.ylabel('Standard Deviation')
    plt.ylim([np.amin(alpha_var),np.amax(alpha_var)])
    if len(sensor_loc)>0: # if there is any sensor
        plt.title('Sensor location (shown as green line)= {} m \nSensor is at node  {}.'
                  .format(sensor_loc, sensor_loc_n), fontsize=14, loc="left")
    plt.show()


## FE Solution

In [10]:
# The only difference is T_ini as input which is an array
def FE(t_start,t_end,delt,Length_c,Length_t,num_el_c,num_el_t,Coords_start,
       air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma,
       a_c,b_c,Ch_c,a_t,b_t,Ch_t,BB,A1,A2,A3,dE1,dE2,dE3,
       Analysis,cri,Element,heat_gen,T_previous,alpha_previous,alpha_dot_previous):  
    
    num_el = num_el_c + num_el_t
    #T_old = np.ones((num_el+1,1))*293
    if Element == 'Linear':
        Nodes, Elements, Coords = Mesh(Coords_start,Length_c,num_el_c,Length_t,num_el_t)
        T_sys = np.copy(T_previous) #np.full((num_el+1,1), T_ini)   
        T_sol = np.copy(T_sys)
        T_sol = np.reshape(T_sys,(num_el+1,1))
        T_sys_new = np.copy(T_sys)
    elif Element == "Nonlinear":
        Nodes, Elements, Coords = Mesh3(Coords_start,Length_c,num_el_c,Length_t,num_el_t)
        T_sys = np.copy(T_previous)# np.full((2*num_el+1,1), T_ini)
        T_sol = np.copy(T_sys)
        T_sol = np.reshape(T_sys,(2*num_el+1,1))
        T_sys_new = np.copy(T_sys)

    alpha_sys = np.copy(alpha_previous)
    alpha_sol = np.copy(alpha_sys)
    alpha_sol = np.reshape(alpha_sys,(num_el,1))
    alpha_sys_new = np.copy(alpha_sys)    
    
    alpha_dot_sys = np.copy(alpha_dot_previous ) 
    alpha_dot_sol = np.copy(alpha_dot_sys)
    alpha_dot_sol = np.reshape(alpha_dot_sys,(num_el,1))
    alpha_dot_sys_new = np.copy(alpha_dot_sys)  
    
       
    for t in np.arange(t_start+delt,t_end+delt,delt): # loop for each time step
              
        for el in range(1,num_el+1):
            if el > num_el_t:
                T_el = 0.5*(T_sys[el-1] + T_sys[el])
                alpha_sys_new[el-1] = alpha_sys[el-1] + delt * alpha_dot_sys[el-1]
                alpha_dot_sys_new[el-1] = alpha_dot_func(alpha_sys_new[el-1],T_el,BB,A1,A2,A3,dE1,dE2,dE3)
            else:
                alpha_dot_sys_new[el-1] = 0
  
        T_a = T_air(t,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma) # current air temperature
        norm = 1000
        # loop for each iteration (meaningful for implicit only)
        while norm>cri:
            if Element == 'Linear':
                K_sys = np.zeros([num_el+1,num_el+1])
                C_sys = np.zeros([num_el+1,num_el+1])
                C_lumped_sys = np.zeros([num_el+1,num_el+1])
                F_sys = np.zeros(num_el+1)
            elif Element == "Nonlinear":
                K_sys = np.zeros([2*num_el+1,2*num_el+1])
                C_sys = np.zeros([2*num_el+1,2*num_el+1])
                C_lumped_sys = np.zeros([2*num_el+1,2*num_el+1])
                F_sys = np.zeros(2*num_el+1)

            # creating global matrices from elements matrices
            for el in range(1,num_el+1):
                if Element == 'Linear':
                    # Element arrays
                    K_el, C_el, C_lumped_el, F_el = KCF(el,a_c,b_c,a_t,b_t,alpha_dot_sys_new[el-1],Coords)
                    # Assembly
                    K_sys, C_sys, C_lumped_sys, F_sys = Assemble(el,K_el,C_el,C_lumped_el,F_el,K_sys,C_sys,C_lumped_sys,F_sys)
                elif Element == "Nonlinear": 
                    # Element arrays
                    K_el, C_el, C_lumped_el, F_el = KCF3(el,a_c,b_c,a_t,b_t,alpha_dot_sys_new[el-1],Coords)
                    # Assembly
                    K_sys, C_sys, C_lumped_sys, F_sys = Assemble3(el,K_el,C_el,C_lumped_el,F_el,K_sys,C_sys,C_lumped_sys,F_sys)
            if heat_gen == 'No':
                F_sys *= 0

            # Boundary condition
            F_sys[0] += Ch_t * (T_a - T_sys[0]) # at tool boundary
            if Element == 'Linear':
                F_sys[num_el] += Ch_c * (T_a - T_sys[num_el])
                F_sys.shape = (num_el+1,1)
            elif Element == "Nonlinear": 
                F_sys[2*num_el] += Ch_c * (T_a - T_sys[2*num_el])
                F_sys.shape = (2*num_el+1,1)

            # Solution
            if Analysis == 'Forward':
                T_dot = sp.linalg.solve(C_lumped_sys, F_sys - K_sys.dot(T_sys)) 
                T_sys_new = T_sys + delt * T_dot # updating results
                norm = 0
            elif Analysis == 'Backward':
                # See S. Amini Niaki thesis, Equations A-6 to A-13
                LHS = C_sys + delt * K_sys
                RHS = delt * F_sys + C_sys.dot(T_sys)
                T_sys_old = T_sys_new
                T_sys_new = sp.linalg.solve(LHS, RHS)  # updating results
                norm = np.sum(np.square(T_sys_new-T_sys_old)) 
        #clear_output(wait=True)
        #print ("progress is : {}%".format(round((t-t_start)/(t_end-t_start)*100,1)))    
        T_sys = T_sys_new
        T_sol = np.append(T_sol, T_sys, axis=1)
           
        alpha_sys = alpha_sys_new
        alpha_sol = np.append(alpha_sol, alpha_sys, axis=1)
        alpha_dot_sys = alpha_dot_sys_new
        alpha_dot_sol = np.append(alpha_dot_sol, alpha_dot_sys, axis=1)
        
        
    return T_sol, Coords, alpha_sol, alpha_dot_sol

## Example 1 - One Hold Temperature Cycle

In [11]:
# material properties
rho_c = 1463 # composites density (kg/m3) 
# --> 1463 for AS4/3501-6 composites (https://pdfs.semanticscholar.org/f069/9fb46a1958f250cc748a673e5a6b8e1910c6.pdf)
#--> 1790 for AS4 carbon (https://www.900gpa.com/en/product/fiber/CF_001EF245BC?u=metric)
k_c = 0.65 # composites thermal conductivity (W/m K) 
# --> 0.65 for AS4/3501-6 composites (https://pdfs.semanticscholar.org/f069/9fb46a1958f250cc748a673e5a6b8e1910c6.pdf)
#--> 6.83 for AS4 carbon (https://www.900gpa.com/en/product/fiber/CF_001EF245BC?u=metric)
Cp_c = 1200 # composite specific heat capacity (J/kg K) 
# --> 1200 for AS4/3501-6 composites (https://pdfs.semanticscholar.org/f069/9fb46a1958f250cc748a673e5a6b8e1910c6.pdf)
# --> 1300 for AS4 Carbon (https://www.researchgate.net/figure/Specific-heat-capacity-of-AS4-carbon-fiber-PES-matrix-and-CF-PES-tape-fiber-volume_fig6_320801788)
rho_r = 1256 # resin density  (kg/m3), 
# -->1256 for 3501-6 (https://www.researchgate.net/figure/3-Properties-of-Hexcel-3501-6-Epoxy-Resin-17_tbl3_267585693)
H_r = 400e3 # resin heat of reasction per unit mass (J / kg)
# --> 400*1000  for 3501-6 (https://books.google.ca/books?id=p__RBQAAQBAJ&pg=PA478&lpg=PA478&dq=resin+3501-6+heat+reaction+per+unit+mass&source=bl&ots=yzGE-Cu-Fo&sig=ACfU3U07FEurjhNeAVzwOKofNp-Y_zYDdw&hl=en&sa=X&ved=2ahUKEwjut6Lx2OboAhUMrp4KHf90BkAQ6AEwAHoECAsQLA#v=onepage&q=resin%203501-6%20heat%20reaction%20per%20unit%20mass&f=false)
nu_r = 0.33 # resin volume fraction in composite material 
# --> 0.33
h_c = 120; # convection heat trasnfer coefficient (W/ m2 K)
# --> 120 in autoclave (https://www.semanticscholar.org/paper/HEAT-TRANSFER-COEFFICIENT-DISTRIBUTION-INSIDE-AN-Slesinger-Shimizu/b61dfa6b4811edb51b003e43cc61088f0d13e348)

a_c =  k_c/(rho_c*Cp_c)
b_c =  rho_r*H_r*nu_r/(rho_c*Cp_c)
Ch_c = h_c/k_c*a_c; 

# cure kenetic
# Table 5.2 of S. Amini Niaki thesis for 3501-6
A1 = 3.5017e7
A2 = -3.3567e7
A3 = 3.2667e3
dE1 = 80700
dE2 = 77800
dE3 = 56600
# Table 5.2 of S.A. Niaki thesis for 3501-6  
BB = 0.47

# tool properties
rho_t = 8150; # tool density (kg/m3) 
# -->  ~ 8150 for Invar (https://www.azom.com/properties.aspx?ArticleID=515)
k_t = 13; # tool thermal conductivity (W/m K) 
# --> ~13 for Invar (https://www.azom.com/properties.aspx?ArticleID=515)
Cp_t = 510; # tool specific heat capacity (J/kg K) 
# --> ~ 510 for Invar (https://www.azom.com/properties.aspx?ArticleID=515)
h_t = 100;
a_t =  k_t/(rho_t*Cp_t);
b_t =  0;
Ch_t = h_t/k_t*a_t; 

# geometry
Coords_start = 0 # first node x coordinate
sensor_loc_typ = "node" # "node" for ndoe numbers, "loc" for locations in meter
sensor_loc_list = [2,7] # node numbers or location of snesors (m)
# composite
Length_c = 0.030 # rod length (m)
num_el_c = 7 # number of elements
# tool
Length_t = 0.015 # tool length (m)
num_el_t = 5 # number of elements

t_start = 0 # start time (seconds)
t_end = 240*60 # end time (seconds)
delt = 1 # time step (seconds)
n = int(int(t_end-t_start)/delt + 1) # number of states


# analysis type
Analysis = 'Forward';  # 'Backward' or 'Forward', Backward is Implicit Euler w/ Newton Raphson, Forward is Expicit Euler
cri = 0.01 # convergence criteria value for Implicit analysis
Element_type = 'Linear' # 'Linear' or 'Nonlinear'

# heat generation switch
heat_gen = 'Yes' # 'Yes' or 'No'

# air temperautre
air_temp_type = 'OneHold' # 'Constant', 'ConstantRate', 'OneHold'
T_start = 20+273 # start air temperature (K)
T_const = 180+273 # air consat temperate (for 'Constat' type) (K)
T_rate = 0.5 # air temperature increase rate (for 'Constant_rate' type)
T_hold = 170+273 # air hold temperate (for 'OneHold' type) (K)
th1 = 70*60 # time for start of hold (for 'OneHold' type) (seconds)
th2 = 170*60 # time for end of hold (for 'OneHold' type) (seconds)

num_el = num_el_c + num_el_t
T_air_sigma = 0
T_ini = np.ones((num_el+1,1))* T_air(0,air_temp_type,T_start,
                                     T_hold,T_const,T_rate,th1,th2,0) # initital temperature of material

alpha_ini = np.zeros((num_el,1))
alpha_dot_ini = np.zeros((num_el,1))

In [12]:
T_true, Coords, alpha_true, alpha_dot_true, = FE(t_start,t_end,delt,Length_c,Length_t,num_el_c,num_el_t,Coords_start,air_temp_type,
                    T_start,T_hold,T_const,T_rate,th1,th2,0,
                                                 a_c,b_c,Ch_c,a_t,b_t,Ch_t,BB,A1,A2,A3,dE1,dE2,dE3,
                    Analysis,cri,Element_type,heat_gen,T_ini,alpha_ini,alpha_dot_ini)


In [13]:
interactive(lambda t=0: plot_Temp(t,t_start,delt,T_ini,Analysis,Coords,T_true,air_temp_type,
                                  num_el_c,num_el_t,a_c,b_c,Ch_c),
            t=(t_start,t_end,(t_end-t_start)/10))

interactive(children=(FloatSlider(value=0.0, description='t', max=14400.0, step=1440.0), Output()), _dom_class…

In [14]:
interactive(lambda element=1: plot_T_alpha_el(element,t_start,t_end+delt,delt,T_true,alpha_true,alpha_dot_true,air_temp_type),
            element=range(1,num_el+1))

interactive(children=(Dropdown(description='element', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), value=1…

## Example 2 - Constant Ambient Temperature

In [15]:
# material properties
rho_c = 1463 # composites density (kg/m3) 
# --> 1463 for AS4/3501-6 composites (https://pdfs.semanticscholar.org/f069/9fb46a1958f250cc748a673e5a6b8e1910c6.pdf)
#--> 1790 for AS4 carbon (https://www.900gpa.com/en/product/fiber/CF_001EF245BC?u=metric)
k_c = 0.65 # composites thermal conductivity (W/m K) 
# --> 0.65 for AS4/3501-6 composites (https://pdfs.semanticscholar.org/f069/9fb46a1958f250cc748a673e5a6b8e1910c6.pdf)
#--> 6.83 for AS4 carbon (https://www.900gpa.com/en/product/fiber/CF_001EF245BC?u=metric)
Cp_c = 1200 # composite specific heat capacity (J/kg K) 
# --> 1200 for AS4/3501-6 composites (https://pdfs.semanticscholar.org/f069/9fb46a1958f250cc748a673e5a6b8e1910c6.pdf)
# --> 1300 for AS4 Carbon (https://www.researchgate.net/figure/Specific-heat-capacity-of-AS4-carbon-fiber-PES-matrix-and-CF-PES-tape-fiber-volume_fig6_320801788)
rho_r = 1256 # resin density  (kg/m3), 
# -->1256 for 3501-6 (https://www.researchgate.net/figure/3-Properties-of-Hexcel-3501-6-Epoxy-Resin-17_tbl3_267585693)
H_r = 400e3 # resin heat of reasction per unit mass (J / kg)
# --> 400*1000  for 3501-6 (https://books.google.ca/books?id=p__RBQAAQBAJ&pg=PA478&lpg=PA478&dq=resin+3501-6+heat+reaction+per+unit+mass&source=bl&ots=yzGE-Cu-Fo&sig=ACfU3U07FEurjhNeAVzwOKofNp-Y_zYDdw&hl=en&sa=X&ved=2ahUKEwjut6Lx2OboAhUMrp4KHf90BkAQ6AEwAHoECAsQLA#v=onepage&q=resin%203501-6%20heat%20reaction%20per%20unit%20mass&f=false)
nu_r = 0.33 # resin volume fraction in composite material 
# --> 0.33
h = 120; # convection heat trasnfer coefficient (W/ m2 K)
# --> 120 in autoclave (https://www.semanticscholar.org/paper/HEAT-TRANSFER-COEFFICIENT-DISTRIBUTION-INSIDE-AN-Slesinger-Shimizu/b61dfa6b4811edb51b003e43cc61088f0d13e348)

a =  k_c/(rho_c*Cp_c)
b =  rho_r*H_r*nu_r/(rho_c*Cp_c)
Ch = h/k_c*a; 

# cure kenetic
# Table 5.2 of S. Amini Niaki thesis for 3501-6
A1 = 3.5017e7
A2 = -3.3567e7
A3 = 3.2667e3
dE1 = 80700
dE2 = 77800
dE3 = 56600
# Table 5.2 of S.A. Niaki thesis for 3501-6  
BB = 0.47

# tool properties
rho_t = 8150; # tool density (kg/m3) 
# -->  ~ 8150 for Invar (https://www.azom.com/properties.aspx?ArticleID=515)
k_t = 13; # tool thermal conductivity (W/m K) 
# --> ~13 for Invar (https://www.azom.com/properties.aspx?ArticleID=515)
Cp_t = 510; # tool specific heat capacity (J/kg K) 
# --> ~ 510 for Invar (https://www.azom.com/properties.aspx?ArticleID=515)
h_t = 100;
a_t =  k_t/(rho_t*Cp_t);
b_t =  0;
Ch_t = h_t/k_t*a_t; 

# geometry
Coords_start = 0 # first node x coordinate
sensor_loc_typ = "node" # "node" for ndoe numbers, "loc" for locations in meter
sensor_loc_list = [] # node numbers or location of snesors (m)
# composite
Length_c = 0.030 # rod length (m)
num_el_c = 10 # number of elements
# tool
Length_t = 0.015 # rod length (m)
num_el_t = 5 # number of elements

t_start = 0 # start time (seconds)
t_end = 220*60 # end time (seconds)
delt = 1 # time step (seconds)
n = int(int(t_end-t_start)/delt + 1) # number of states

# analysis type
Analysis = 'Forward';  # 'Backward' or 'Forward', Backward is Implicit Euler w/ Newton Raphson, Forward is Expicit Euler
cri = 0.01 # convergence criteria value for Implicit analysis
Element_type = 'Linear' # 'Linear' or 'Nonlinear'

# heat generation switch
heat_gen2 = 'Yes' # 'Yes' or 'No'

# air temperautre
air_temp_type2 = 'Constant' # 'Constant', 'ConstantRate', 'OneHold'
T_start = 20+273 # start air temperature (K)
T_const = 180+273 # air consat temperate (for 'Constat' type) (K)
T_rate = 0.5 # air temperature increase rate (for 'Constant_rate' type)
T_hold = 170+273 # air hold temperate (for 'OneHold' type) (K)
th1 = 70*60 # time for start of hold (for 'OneHold' type) (seconds)
th2 = 170*60 # time for end of hold (for 'OneHold' type) (seconds)

num_el = num_el_c + num_el_t
T_air_sigma = 0
T_ini2 = np.ones((num_el+1,1))* (20+273)

alpha_ini = np.zeros((num_el,1))
alpha_dot_ini = np.zeros((num_el,1))

In [16]:
T_true2, Coords2, alpha_true2, alpha_dot_true2, = FE(t_start,t_end,delt,Length_c,Length_t,num_el_c,num_el_t,Coords_start,
                                                     air_temp_type2,T_start,T_hold,T_const,T_rate,th1,th2,0,
                                                     a_c,b_c,Ch_c,a_t,b_t,Ch_t,BB,A1,A2,A3,dE1,dE2,dE3,
                                                     Analysis,cri,Element_type,heat_gen2,T_ini2,alpha_ini,alpha_dot_ini)


In [17]:
interactive(lambda t=0: plot_Temp(t,t_start,delt,T_ini2,Analysis,Coords2,T_true2,air_temp_type2,
                                  num_el_c,num_el_t,a_c,b_c,Ch_c),
            t=(t_start,t_end,(t_end-t_start)/10))

interactive(children=(FloatSlider(value=0.0, description='t', max=13200.0, step=1320.0), Output()), _dom_class…

In [18]:
interactive(lambda element=1: plot_T_alpha_el(element,t_start,t_end+delt,delt,T_true2,alpha_true2,alpha_dot_true2,air_temp_type2),
            element=range(1,num_el+1))

interactive(children=(Dropdown(description='element', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, …